In [1]:
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

# ファイルを読み込む
with open("test.ssh", "rb") as f:
    data = f.read()

# 32バイトのキーを生成
key = get_random_bytes(32)

# AES暗号器を作成
cipher = AES.new(key, AES.MODE_CTR)

# ファイルを暗号化
encrypted_data = cipher.encrypt(data)

# 暗号化したファイルを書き込む
with open("test.ssh.enc", "wb") as f:
    f.write(encrypted_data)

# 復号化用にnonce（初期値）を取得
nonce = cipher.nonce

# 復号用のAES暗号器を作成
decipher = AES.new(key, AES.MODE_CTR, nonce=nonce)

# ファイルを復号化
decrypted_data = decipher.decrypt(encrypted_data)

# 復号化したファイルを書き込む
with open("test2.ssh", "wb") as f:
    f.write(decrypted_data)

In [2]:
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes
from Cryptodome.Protocol.KDF import PBKDF2

# ファイルを読み込む
with open("test.ssh", "rb") as f:
    data = f.read()

# パスワードとsaltを入力
password = "test1"
salt = "test2"

# パスワードとsaltから32バイトのキーを生成（PBKDF2関数を使用）
key = PBKDF2(password, salt, dkLen=32)

# AES暗号器を作成（CTRモード）
cipher = AES.new(key, AES.MODE_CTR)

# ファイルを暗号化
encrypted_data = cipher.encrypt(data)

# 暗号化したファイルを書き込む
with open("test.ssh.enc", "wb") as f:
    f.write(encrypted_data)

# 復号化用にnonce（初期値）を取得
nonce = cipher.nonce

# 復号用のAES暗号器を作成
decipher = AES.new(key, AES.MODE_CTR, nonce=nonce)

# ファイルを復号化
decrypted_data = decipher.decrypt(encrypted_data)

# 復号化したファイルを書き込む
with open("test3.ssh", "wb") as f:
    f.write(decrypted_data)

In [26]:
import subprocess

# subprocess.run("ls", shell=True, capture_output=True, text=True).stdout
subprocess.run('dir', capture_output=True, text=True, shell=True)

CompletedProcess(args='dir', returncode=0, stdout=' Volume in drive E is SSD_1TB \n Volume Serial Number is 4E8B-5EBC\n\n Directory of e:\\Projects\\research\\DecentralizedMetaverse\\decentralized-metaverse-unity\\py-metaverse\n\n02/26/2023  11:55 AM    <DIR>          .\n02/26/2023  11:55 AM    <DIR>          ..\n02/25/2023  11:14 PM             1,555 converter.py\n02/25/2023  11:59 PM             1,041 encrypt.py\n02/26/2023  11:46 AM                58 main.py\n02/22/2023  10:14 PM               162 QmazK7rziw12X41D2ytdy93rk9hqFW9qZTKzX1HGThHvzp.yaml\n02/26/2023  10:55 AM                97 qr_code.py\n02/25/2023  11:16 PM             3,271 test.ipynb\n02/26/2023  11:55 AM               105 test.py\n02/26/2023  11:32 AM                62 test.ssh\n02/25/2023  11:16 PM                43 test.ssh.enc\n02/25/2023  11:14 PM                43 test2.ssh\n02/25/2023  11:16 PM                43 test3.ssh\n              11 File(s)          6,480 bytes\n               2 Dir(s)  773,593,481,216 b

In [3]:
filename = "test.ssh.enc"

idx = filename.rfind(".")
filename[0:idx]

'test.ssh'

In [ ]:
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes
from Cryptodome.Protocol.KDF import PBKDF2
from cryptography.hazmat.primitives import padding
import hashlib

def encrypt(filename, password, salt):
    iterations = 10000
    # ファイルを読み込む
    with open(filename, "rb") as f:
        data = f.read()
        
    password = password.encode('utf-8')
    salt = salt.encode('utf-8')
    
    # PBKDF2で鍵と初期化ベクトルを生成（32バイトと16バイト）
    key = hashlib.pbkdf2_hmac("sha256", password, salt, iterations, 32)
    iv = hashlib.pbkdf2_hmac("sha256", password, salt, iterations, 16)

    # AES暗号器を作成
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # データの長さが16バイトの倍数になるようにパディングする
    padder = padding.PKCS7(128).padder()
    data = padder.update(data) + padder.finalize()
    
    # ファイルを暗号化
    encrypted_data = cipher.encrypt(data)

    # 暗号化したファイルを書き込む
    with open(f"{filename}.enc", "wb") as f:
        f.write(encrypted_data)
        

def decrypt(filename, password, salt):
    iterations = 10000
    
    # ファイルを読み込む
    with open(filename, "rb") as f:
        encrypted_data = f.read()
        
    # パディングを取り除く
    unpadder = padding.PKCS7(128).unpadder()
    data = unpadder.update(data) + unpadder.finalize()
        
    password = password.encode('utf-8')
    salt = salt.encode('utf-8')
        
    # PBKDF2で鍵と初期化ベクトルを生成（32バイトと16バイト）
    key = hashlib.pbkdf2_hmac("sha256", password, salt, iterations, 32)
    iv = hashlib.pbkdf2_hmac("sha256", password, salt, iterations, 16)

    # パスワードとsaltから32バイトのキーを生成（PBKDF2関数を使用）
    # key = PBKDF2(password, salt, dkLen=32)

    # AES暗号器を作成
    decipher = AES.new(key, AES.MODE_CBC, iv)

    # ファイルを復号化
    decrypted_data = decipher.decrypt(encrypted_data)
    
    decrypted_data = decrypted_data.rstrip(b"\x00")

    # 復号化したファイルを書き込む
    idx = filename.rfind(".")
    filename = filename[0:idx]
    with open(filename, "wb") as f:
        f.write(decrypted_data)